# Elaborazione parco veicoli circolante

Analisi dell'evoluzione del mercato delle automobili endotermiche ed elettrificate:  
trend evolutivi ed impatto sulle politiche economiche e fiscali

Importazione dataset, data preparing, data cleaning

In [ ]:
#monto google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
# importa il dataset
pth = '/content/drive/MyDrive/Python_Files/'
df = pd.read_excel(pth+'Parco Veicoli.xlsx')

#per rimuovere eventuali spazi vuoti iniziali o finali dai nomi delle colonne in un DataFrame di pandas:
df.columns = df.columns.str.strip()

#cancella le colonne relative ad AL e ND che non sono definite
df.drop(labels=['AL', 'ND'], axis=1, inplace=True)

#sostituisci la stringa ' -   ' con lo 0; dopodichè converti i valori in valori numerici
df.replace(' -   ', '0', inplace=True)
df.iloc[:,3:] = df.iloc[:,3:].applymap(pd.to_numeric)

<ipython-input-2-64865fcca398>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.iloc[:,3:] = df.iloc[:,3:].applymap(pd.to_numeric)


Elaboro il dataset raggruppando i dati per ENDOTERM e ELETTRIC

In [ ]:
# raggruppo i veicoli con motore endotermico nella colonna "Endoterm" e i veicoli con motore elettrico o ibrido nella colonna "Elettric"
endo = ['BE','BG', 'BM', 'GA', 'GG', 'ME']
elet = ['EL', 'IB', 'IG']
df['Endoterm']=df[endo].sum(axis=1)
df['Elettric']=df[elet].sum(axis=1)

#cancello le altre colonne del dataset iniziale
df.drop(labels=endo+elet, axis=1, inplace=True)

Integrazione del dataset del parco veicoli con il dataset della poopolazione maggiorenne

In [ ]:
#importo il dataset della popolazione maggiorenne
df_pop = pd.read_excel(pth+'Pop_maggiorenne.xlsx')

#faccio il merge tra il dataframe elaborato e il dataframe della popolazione maggiorenne aggiungendo solo la colonna "maggiorenni"
df_merged = df.merge(df_pop[['Regione', 'Anno', 'Maggiorenni']], on=['Regione', 'Anno'], how='inner')

#aggiungo le colonne di calcolo relative all'indice veicolo procapite endotermico e elettrificato
df_merged['endo_pop']=df_merged['Endoterm']/df_merged['Maggiorenni']
df_merged['elet_pop']=df_merged['Elettric']/df_merged['Maggiorenni']

Salvataggio ed esportazione del dataset elaborato da usare per la data visualization in Microsoft PowerBI

In [ ]:
# salvo il dataset elaborato in un file excel nominato "parco veicoli_pwrBI.xlsx"
with pd.ExcelWriter('parco veicoli_pwrBI.xlsx') as writer:
  df_merged.to_excel(writer, sheet_name='PoerBI')

# CLUSTER ANALYSIS

Applicazione di algoritmi di CLUSTER ANALYSIS per l'individuazione dei due cluster di regioni


In [ ]:
# importo le libreire di riferimento di SkiKitLearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# imposto come datset per la cluster analysis il risultato dell'elaborazione della sezione "Elaborazione parco veicoli circolante"
df_cluster = df_merged

# Raggruppare i dati per 'Anno' e 'Regione', calcolando la media dei valori di 'endo_pop' e 'elet_pop'
region_year_data = df_cluster.groupby(['Anno', 'Regione'])[['endo_pop', 'elet_pop']].mean().reset_index()

1. Creo una funziona che ha come input una lista di regioni da clusterizzare e come output il dataset filtrato per le stesse e clusterizzato

In [ ]:
def clus(regioni):
  filtered_data = region_year_data[region_year_data['Regione'].isin(regioni)]
  # Inizializzare una lista per salvare i risultati dei cluster per ogni anno con 2 cluster
  results_clusters_filtered = []
  final_results_2_clusters_filtered =[]
  # Inizializzare lo scaler per standardizzare i dati
  scaler = StandardScaler()

  # Creare i cluster per ogni anno con 2 cluster escludendo le regioni specificate
  for year in filtered_data['Anno'].unique():
      # Filtrare i dati per l'anno corrente
      year_data = filtered_data[filtered_data['Anno'] == year]

      # Standardizzare i dati per l'anno corrente
      year_data_scaled = scaler.fit_transform(year_data[['endo_pop', 'elet_pop']])

      # Eseguire il clustering K-means con 2 cluster sui dati delle regioni per l'anno corrente
      kmeans_year_2 = KMeans(n_clusters=2, random_state=42)
      year_clusters_2 = kmeans_year_2.fit_predict(year_data_scaled)

      # Aggiungere i risultati dei cluster al dataframe per l'anno corrente
      year_data['Cluster'] = year_clusters_2

      # Aggiungere i risultati alla lista
      results_clusters_filtered.append(year_data)

  # Concatenare i risultati per tutti gli anni con 2 cluster
  final_results_2_clusters_filtered = pd.concat(results_clusters_filtered, ignore_index=True)
  return final_results_2_clusters_filtered

2. Eseguo la cluster analysis prima tra tutte le regioni escluse le 2 outlier (Valle d'Aosta e Trentino Alto Adige) e successivamente, per ogni cluster individuato, eseguo una clusterizzazione ulteriore per affinare la qualità della cluster analysis

In [ ]:
# Filtrare i dati per escludere le regioni 'Valle D'Aosta' e 'Trentino Alto Adige'
regioni =['UMBRIA','SICILIA', 'SARDEGNA', 'PUGLIA', 'MOLISE', 'MARCHE', 'CAMPANIA', 'CALABRIA', 'BASILICATA', 'ABRUZZO','VENETO', 'TOSCANA', 'PIEMONTE', 'LOMBARDIA', 'LIGURIA', 'LAZIO', 'FRIULI VENEZIA GIULIA', 'EMILIA ROMAGNA']

# prima cluster analysis e individuazione dei 2 cluster
df_clus= clus(regioni)

# clusA contiene la lista delle regioni del cluster A e clusB quella delle regioni del cluster B
clusA= df_clus[(df_clus['Anno'] ==2023)&(df_clus['Cluster'] == 0)]['Regione'].unique()
clusB= df_clus[(df_clus['Anno'] ==2023)&(df_clus['Cluster'] == 1)]['Regione'].unique()

# miglioro la cluster analysis per il cluster A (riducendo il numero di regioni di A)
df_clus= clus(clusA)
clusA1= df_clus[(df_clus['Anno'] ==2023)&(df_clus['Cluster'] == 0)]['Regione'].unique()
clusA2= df_clus[(df_clus['Anno'] ==2023)&(df_clus['Cluster'] == 1)]['Regione'].unique()
if len(clusA1)>len(clusA2):
  clusA= clusA1
else:
   clusA=clusA2
print(clusA) # risultato della cluster analysis: CLUSTER A

# miglioro la cluster analysis per il cluster B riducendo il numero di regioni di B)
df_clus= clus(clusB)
clusB1= df_clus[(df_clus['Anno'] ==2023)&(df_clus['Cluster'] == 0)]['Regione'].unique()
clusB2= df_clus[(df_clus['Annorisulta'] ==2023)&(df_clus['Cluster'] == 1)]['Regione'].unique()
if len(clusB1)>len(clusB2):
  clusB= clusB1
else:
   clusB=clusB2
print(clusB) # risultato della cluster analysis: CLUSTER B

['EMILIA ROMAGNA' 'FRIULI VENEZIA GIULIA' 'LAZIO' 'LOMBARDIA' 'PIEMONTE'
 'TOSCANA' 'VENETO']
['BASILICATA' 'CALABRIA' 'CAMPANIA' 'MOLISE' 'PUGLIA' 'SARDEGNA' 'SICILIA']
